In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# *Text Reduction*
___
#### This project aims to reduce texts from the internet using NLP techniques.

# Packages :

In [2]:
# !pip install beautifulsoup4

In [86]:
import nltk
from nltk.tokenize import  word_tokenize,sent_tokenize
import re
import requests
from bs4 import BeautifulSoup
import string

stopwords = nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [87]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("href=")
stopwords.append("class=")
stopwords.append("id=")
stopwords.append("title=")

In [88]:
ponc = string.punctuation + "  '' ,\"\""

In [18]:
ponc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~  \'\' ,""'

## Get articals:

In [89]:
url = "https://en.wikipedia.org/wiki/Liquid_state_machine"

In [90]:
def weghts_extracting(url):
    
    responce = requests.get(url)
    
    html = responce.text
    html_obj = BeautifulSoup(html,'html.parser')
    
    # find all paragraphs
    all_paragraph = html_obj.find_all("p")
    
    # remove html tags
    pattren = "\<\/?.{1,4}\>"
    all_paragraph = [re.sub(pattren,"",str(i)) for i in all_paragraph]
    
    clean_txt = all_paragraph
    
    # tokenize words
    all_paragraph = [word_tokenize(txt.lower()) for txt in all_paragraph]
    
    # remove punctuation
    all_paragraph = [[word for word in txt if word not in ponc] for txt in all_paragraph]
    
    # remove stopwords
    all_paragraph = [[word for word in txt if word not in stopwords] for txt in all_paragraph]
    
    # words freq
    word_fr = {}

    for txt in all_paragraph:
        for word in txt:
            if word in word_fr.keys():
                word_fr[word] += 1
            else:
                word_fr[word] = 1
                
    # get max freq
    max_val = max(word_fr.values())
    
    # normalizeation
    word_fr_nrml = {}
    for key,val in word_fr.items():
        word_fr_nrml[key] = val/max_val
        
    return clean_txt , word_fr_nrml


In [108]:
clean_text, word_fr = weghts_extracting(url)

In [168]:
def text_reduction(clean_text,word_frq,threshold=3):
    
    tokenized = [sent_tokenize(txt) for txt in clean_text]
    
    sents_res = {}
    for txt in tokenized:
        for sent in txt:
            sents_res[sent] = 0
            for word in word_tokenize(sent):
                if word in word_fr_nrml.keys():
                    sents_res[sent] += word_fr_nrml[word]
                    
    sents_res_keys = []
    sents_res_values = np.array([], dtype="float64")

    for k,v in sents_res.items():
        sents_res_keys.append(k)
        sents_res_values = np.append(sents_res_values, v)
        
    for cnt in range(len(sents_res)):
        if sents_res_values[cnt] >=5:
            print(sents_res_keys[cnt])

In [144]:
sents_res = {}
for txt in tokenized:
    for sent in txt:
        sents_res[sent] = 0
        for word in word_tokenize(sent):
            if word in word_fr_nrml.keys():
                sents_res[sent] += word_fr_nrml[word]


In [162]:
sents_res_keys = []
sents_res_values = np.array([], dtype="float64")

for k,v in sents_res.items():
    sents_res_keys.append(k)
    sents_res_values = np.append(sents_res_values, v)
    

In [166]:
for cnt in range(len(sents_res)):
    if sents_res_values[cnt] >=5:
        print(sents_res_keys[cnt])

A liquid state machine (LSM) is a type of <a href="/wiki/Reservoir_computing" title="Reservoir computing">reservoir computer that uses a <a href="/wiki/Spiking_neural_network" title="Spiking neural network">spiking neural network.
The <a href="/wiki/Recurrent_neural_network" title="Recurrent neural network">recurrent nature of the connections turns the time varying input into a <a class="mw-redirect" href="/wiki/Spatio-temporal_pattern" title="Spatio-temporal pattern">spatio-temporal pattern of activations in the network nodes.
Given a <a class="mw-redirect" href="/wiki/Typewriting_monkeys" title="Typewriting monkeys">large enough variety of such nonlinear functions, it is theoretically possible to obtain linear combinations (using the read out units) to perform whatever mathematical operation is needed to perform a certain task, such as <a href="/wiki/Speech_recognition" title="Speech recognition">speech recognition or <a href="/wiki/Computer_vision" title="Computer vision">computer v